In [2]:
import pulp
import pandas as pd
import numpy as np
from scipy.spatial import distance_matrix
import random
import gurobipy as grb

from numpy.core.fromnumeric import transpose
from scipy.spatial.distance import minkowski
from custom_parser import parse_file
%load_ext autoreload
%autoreload 2

In [47]:
df = parse_file('testcase0.def')

In [13]:
# df = pd.read_csv('https://raw.githubusercontent.com/mashazya/Datathon2022/main/data/parsed_example_input.csv', sep=',')

In [48]:
driver_centr_x = int((df[df.driver_type != 0].x).mean())
driver_centr_y = int((df[df.driver_type != 0].y).mean())

In [49]:
drivers = df[df['driver_type'] != 0]
df = df.drop([i for i in range(32)])

In [50]:
df.loc[-1] = ['depot', driver_centr_x, driver_centr_y, 1]
df.index = df.index + 1  # shifting index
df = df.sort_index().reset_index(drop=True)

In [51]:
n = len(df[['x', 'y']].values.tolist())
L = n
K = 1
salesmen = 16

def distance(points, i, j):
    dx = abs(points[i][0] - points[j][0])
    dy = abs(points[i][1] - points[j][1])
    return dx+dy

random.seed(1)
points = df[['x', 'y']].values.tolist()


m = grb.Model()
m.setParam('TimeLimit', 5*60)

# Create variables

vars = {}

for i in range(n):
    for j in range(n):
        vars[i, j] = m.addVar(obj=distance(points, i, j), vtype=grb.GRB.BINARY, name='e_'+str(i)+'_'+str(j))
m.update()

uVars = {}
for i in range(n):
    uVars[i] = m.addVar(lb=K, ub=L, vtype=grb.GRB.INTEGER, name='u_'+str(i))
    vars[i, i].ub = 0
m.update()

m.addConstr(grb.quicksum(vars[0, i] for i in range(1, n)) == salesmen)
m.update()

m.addConstr(grb.quicksum(vars[i, 0] for i in range(1, n)) == salesmen)
m.update()

for j in range(1, n):
    m.addConstr(grb.quicksum(vars[i, j] for i in range(n)) == 1)
m.update()

for i in range(1, n):
    m.addConstr(grb.quicksum(vars[i, j] for j in range(n)) == 1)
m.update()

for i in range(1, n):
    m.addConstr(uVars[i] + (L - 2)*vars[0, i] - vars[i, 0] <= (L - 1))
m.update()

for i in range(1, n):
    m.addConstr(uVars[i] + vars[0, i] + (2 - K)*vars[i, 0] >= 2)
m.update()

for i in range(1, n):
    m.addConstr(vars[0, i] + vars[i, 0] <= 1)
m.update()

for i in range(1, n):
    for j in range(1, n):
        if i != j:
            m.addConstr(uVars[i] - uVars[j] + L*vars[i, j] + (L - 2)*vars[j, i] <= (L - 1))
m.update()

print('Constraints loaded')

# m.write("mtsp.lp")

# m.update()

totVars = dict(list(vars.items())+list(uVars.items()))
m._vars = vars
m._uvars = uVars

m.optimize()

solution = m.getAttr('x', vars)
selected = [(i,j) for i in range(n) for j in range(n) if solution[i,j] > 0.5]

uValues = m.getAttr('x', uVars)
# print("U values: ", uValues)

print('')
# print('Optimal tour: %s' % str(selected))
print(selected)
print('Optimal cost:', m.objVal/16)
print('')

Set parameter TimeLimit to value 300
Constraints loaded
Gurobi Optimizer version 10.0.0 build v10.0.0rc2 (win64)

CPU model: Intel(R) Core(TM) i7-6500U CPU @ 2.50GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads

Optimize a model with 1004002 rows, 1003002 columns and 6008000 nonzeros
Model fingerprint: 0x2427bf5e
Variable types: 0 continuous, 1003002 integer (1002001 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+03]
  Objective range  [1e+03, 9e+06]
  Bounds range     [1e+00, 1e+03]
  RHS range        [1e+00, 1e+03]
Presolve removed 0 rows and 1002 columns (presolve time = 5s) ...
Presolve removed 0 rows and 1002 columns (presolve time = 10s) ...
Presolve removed 0 rows and 1002 columns (presolve time = 15s) ...
Presolve removed 0 rows and 1002 columns (presolve time = 20s) ...
Presolve removed 0 rows and 1002 columns (presolve time = 25s) ...
Presolve removed 0 rows and 1002 columns
Presolve time: 26.03s
P

In [ ]:
route = [None]*16
for i in range(16):
    start = selected[i]
    route[i] = [start[0]]
    while start[1] != 0:
        for s in selected[16:]:
            if s[0] == start[1]:
                route[i].append(s[0])
                start = s
    route[i].append(0)

In [24]:
driver_input = drivers[drivers['driver_type']==1][['x','y']].values
driver_output = drivers[drivers['driver_type']==2][['x','y']].values

In [25]:
mat_dist_in = np.zeros((16,32))
mat_dist_out= np.zeros((16,32))
for d in range(16):
    for r in range(16):
        mat_dist_in[d][r] = minkowski(driver_input[d], df.values[route[r][1]][1:3], 1)
        mat_dist_in[d][r + 16] = minkowski(driver_input[d], df.values[route[r][-2]][1:3], 1)

        mat_dist_out[d][r] = minkowski(driver_output[d], df.values[route[r][1]][1:3], 1)
        mat_dist_out[d][r + 16] = minkowski(driver_output[d], df.values[route[r][-2]][1:3], 1)

In [26]:
for i in range(16):
    pin = np.argmin(mat_dist_in[i])
    if pin < 16:
        #driver inici
        route[pin][0] = i
        mat_dist_in[:,pin] = np.Inf
        mat_dist_in[:,pin+16] = np.Inf

        #driver final
        driv = np.argmin(mat_dist_out[:,pin + 16])
        route[pin][-1] = driv
        mat_dist_out[:,pin+16] = np.Inf
        mat_dist_out[:,pin] = np.Inf
        mat_dist_out[driv,:] = np.Inf
    else:
        #driver inici
        route[pin - 16][-1] = i
        mat_dist_in[:,pin] = np.Inf
        mat_dist_in[:,pin-16] = np.Inf

        #driver final
        driv = np.argmin(mat_dist_out[:,pin - 16])
        route[pin-16][0] = driv
        mat_dist_out[:,pin-16] = np.Inf
        mat_dist_out[:,pin] = np.Inf
        mat_dist_out[driv,:] = np.Inf

        route[pin-16].reverse()

In [35]:
def parse_output(net_name, routes):
    pins = df[df['driver_type'] != 1]
    with open(net_name+'_optimizer.def', 'w') as f:
        for route in routes:
            f.writelines([net_name,'\n'])
            f.writelines(['  ( ', drivers['name_pin'].tolist()[route[0]], ' conn_in )\n'])
            f.writelines(['  ( ', pins['name_pin'].tolist()[route[1]-1], ' conn_out )\n'])
            f.write(';\n')
            for i in range (1,len(route)-2):
                f.writelines([net_name,'\n'])
                f.writelines(['  ( ', pins['name_pin'].tolist()[route[i]-1], ' conn_in )\n'])
                f.writelines(['  ( ', pins['name_pin'].tolist()[route[i+1]-1], ' conn_out )\n'])
                f.write(';\n')

            f.writelines([net_name,'\n'])
            f.writelines(['  ( ', pins['name_pin'].tolist()[route[-2]-1], ' conn_in )\n'])
            f.writelines(['  ( ', drivers['name_pin'].tolist()[route[-1]+16], ' conn_out )\n'])
            f.write(';\n')
        f.close()

In [36]:
parse_output('- TESTCASE 0', route)

In [113]:
#df contains all the pins and the initial avg driver
#drivers contains the drivers
#dist_matrix calculated between avg driver and pins
def calculate_metric(routes, df, drivers):
    sum_length = []
    for r in route:
        l = []

        for i in range(0,len(r)):
            if i == 0:
                driver_in = drivers[drivers['driver_type'] == 1].iloc[r[i]].values[1:3]
                pin = df.loc[r[i+1]].values[1:3]
                l.append(minkowski(driver_in, pin, 1))
            elif i == len(r)-2:
                driver_out = drivers[drivers['driver_type'] == 2].iloc[r[i+1]].values[1:3]
                pin = df.loc[r[i]].values[1:3]
                l.append(minkowski(driver_out, pin, 1))
                break
            else:
                l.append(minkowski(df.loc[r[i]].values[1:3], df.loc[r[i+1]].values[1:3]))
        sum_length.append(sum(l))

    return sum(sum_length) / 16, np.std(sum_length), sum_length


In [ ]:
calculate_metric(route, dist_matrix, df, drivers)